In [433]:
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process 
import pandas as pd
import numpy as np
from collections import defaultdict
import re
import csv

In [434]:
stations_df = pd.read_csv('stations_zip.csv')
stations_df.drop(['LATITUDE', 
                  'LONGITUDE'], axis=1, inplace = True)
for column in stations_df.columns:    
    stations_df.rename(columns={column: column.lower().strip()}, inplace=True)

    
stations_df.sort_values(by = ['name'], inplace = True)
stations_df.head()

,unnamed: 0,name,line,zipcode
0,0,103rd St,ABC,10025-4403
1,1,103rd St,466,10029
2,2,103rd St,1,10025
3,3,103rd St - Corona Plaza,7,11368
4,4,104th St,AS,11419


In [282]:
def get_data(week_nums):
    url = 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt'
    return pd.concat(list(map(lambda x: pd.read_csv(url.format(x)), week_nums)))
weeks = [200606]
turnstile_df = get_data(weeks)


In [287]:
turnstile_df.sort_values(by = ['STATION'], inplace = True)
turnstile_df.LINENAME = turnstile_df.LINENAME.str.replace(' ','')
turnstile_df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
33077,H008,R248,01-00-00,1 AV,L,BMT,06/01/2020,00:00:00,REGULAR,303244,4993334
33074,H008,R248,01-00-00,1 AV,L,BMT,05/31/2020,12:00:00,REGULAR,303242,4993298
33073,H008,R248,01-00-00,1 AV,L,BMT,05/31/2020,08:00:00,REGULAR,303242,4993279
33072,H008,R248,01-00-00,1 AV,L,BMT,05/31/2020,04:00:00,REGULAR,303242,4993272
33071,H008,R248,01-00-00,1 AV,L,BMT,05/31/2020,00:00:00,REGULAR,303242,4993272


***Here's how we're solving it:***  
We create this dictionary, and then construct the same dictionary for the turnstile data.  
Then we run simple comparisons to match up by stop_name and line (the subway line), and then in the stations dictionary, append a field with the turnstyle ID associated with the station ID. Then we simply need to apply that dictionary and it's zipcodes to the turnstile dataframe. AND THEN IT'S LITERALLY DONE.  
We will have the turnstile data with zip-codes!

In [435]:
stations_df.name = stations_df.name.str.replace('rd', '')
stations_df.name = stations_df.name.str.replace('th', '')
stations_df.name = stations_df.name.str.replace('nd', '')
stations_df.name = stations_df.name.str.replace('st', '')
stations_df.line = stations_df.line.str.replace(' ', '')

In [436]:
stations_df.head()

,unnamed: 0,name,line,zipcode
0,0,103 St,ABC,10025-4403
1,1,103 St,466,10029
2,2,103 St,1,10025
3,3,103 St - Corona Plaza,7,11368
4,4,104 St,AS,11419


In [475]:
stations_dict = defaultdict(dict)

for i in stations_df.itertuples():
    stations_dict[i[1]]['stop_name'] = re.sub('[^A-Za-z0-9]+', '', i[2]).lower()
    stations_dict[i[1]]['line'] = i[3]
    stations_dict[i[1]]['zipcode'] = i[4]
    stations_dict[i[1]]['turnstile_station_id'] = ''
    

In [472]:
stations_dict

defaultdict(dict,
            {0: {'stop_name': '103st',
              'line': 'ABC',
              'zipcode': '10025-4403',
              'turnstile_station_id': 'R314'},
             1: {'stop_name': '103st',
              'line': '466',
              'zipcode': '10029',
              'turnstile_station_id': 'R180'},
             2: {'stop_name': '103st',
              'line': '1',
              'zipcode': '10025',
              'turnstile_station_id': 'R191'},
             3: {'stop_name': '103stcoronaplaza',
              'line': '7',
              'zipcode': '11368',
              'turnstile_station_id': 'R208'},
             4: {'stop_name': '104st',
              'line': 'AS',
              'zipcode': '11419',
              'turnstile_station_id': 'R354'},
             5: {'stop_name': '104102sts',
              'line': 'JZ',
              'zipcode': '11418',
              'turnstile_station_id': 'R007'},
             6: {'stop_name': '110st',
              'line': '466',
      

In [295]:
turnstiles_dict = defaultdict(dict)

for i in turnstile_df.itertuples():
    turnstiles_dict[i[2]]['stop_name'] = re.sub('[^A-Za-z0-9]+', '',i[4]).lower()
    turnstiles_dict[i[2]]['line'] = i[5]

In [296]:
turnstiles_dict

defaultdict(dict,
            {'R248': {'stop_name': '1av', 'line': 'L'},
             'R180': {'stop_name': '103st', 'line': '6'},
             'R314': {'stop_name': '103st', 'line': 'BC'},
             'R191': {'stop_name': '103st', 'line': '1'},
             'R208': {'stop_name': '103stcorona', 'line': '7'},
             'R354': {'stop_name': '104st', 'line': 'A'},
             'R007': {'stop_name': '104st', 'line': 'JZ'},
             'R181': {'stop_name': '110st', 'line': '6'},
             'R355': {'stop_name': '111st', 'line': 'A'},
             'R310': {'stop_name': '111st', 'line': '7'},
             'R008': {'stop_name': '111st', 'line': 'J'},
             'R324': {'stop_name': '116st', 'line': '23'},
             'R333': {'stop_name': '116st', 'line': 'BC'},
             'R182': {'stop_name': '116st', 'line': '6'},
             'R159': {'stop_name': '116stcolumbia', 'line': '1'},
             'R009': {'stop_name': '121st', 'line': 'JZ'},
             'R102': {'stop_name': '1

In [476]:
for j in stations_dict:
    print(stations_dict[j])
    for i in turnstiles_dict:
        if turnstiles_dict[i]['stop_name'] == stations_dict[j]['stop_name']):
                if match_lines(i,j):
                    match_lines(i,j)
                    break
    for i in turnstiles_dict:
        if (stations_dict[j]['turnstile_station_id'] =='' and \
        stations_dict[j]['stop_name'] in turnstiles_dict[i]['stop_name']):
                if match_lines(i,j):
                    match_lines(i,j)
                    break
    for i in turnstiles_dict:
        if (stations_dict[j]['turnstile_station_id'] =='' and \
        turnstiles_dict[i]['stop_name'] in stations_dict[j]['stop_name']):
                if match_lines(i,j):
                    match_lines(i,j)
                    break
    for i in turnstiles_dict:
        if (stations_dict[j]['turnstile_station_id'] =='' and \
        fuzz.ratio(turnstiles_dict[i]['stop_name'], stations_dict[j]['stop_name']) > 70):
                if match_lines(i,j):
                    match_lines(i,j)
                    break

            
        def match_lines(i,j):
            match = False
            if len(set(stations_dict[j]['line'])) == len(set(turnstiles_dict[i]['line'])):
                for char in set(stations_dict[j]['line']):
                    if turnstiles_dict[i]['line'].count(char)==0:
                        match = False
                    else:
                        match = True
            elif len(set(stations_dict[j]['line'])) < len(set(turnstiles_dict[i]['line'])):
                for char in set(stations_dict[j]['line']):
                    if turnstiles_dict[i]['line'].count(char)==0:
                        match = False
                    else:
                        station_line_match = True
            elif len(set(stations_dict[j]['line'])) > len(set(turnstiles_dict[i]['line'])):
                for char in set(turnstiles_dict[i]['line']):
                    if stations_dict[j]['line'].count(char)==0:
                        match = False
                    else:
                        match = True
            
            
                        
            if match:
                stations_dict[j]['turnstile_station_id'] = i
                
                print('station ',stations_dict[j]['stop_name'])
                print(stations_dict[j]['line'])
                print('turnstile ',turnstiles_dict[i]['stop_name'])
                print(turnstiles_dict[i]['line'])
                print('----------------------------')
            
            return match
        
for i in stations_dict:
    if stations_dict[i]['turnstile_station_id'] == '':
            stations_dict[i]['turnstile_station_id'] = ''#np.nan

                                
                    
            
            
                            
            

{'stop_name': '103st', 'line': 'ABC', 'zipcode': '10025-4403', 'turnstile_station_id': ''}
station  103st
ABC
turnstile  103st
BC
----------------------------
station  103st
ABC
turnstile  103st
BC
----------------------------
{'stop_name': '103st', 'line': '466', 'zipcode': '10029', 'turnstile_station_id': ''}
station  103st
466
turnstile  103st
6
----------------------------
station  103st
466
turnstile  103st
6
----------------------------
{'stop_name': '103st', 'line': '1', 'zipcode': '10025', 'turnstile_station_id': ''}
station  103st
1
turnstile  103st
1
----------------------------
station  103st
1
turnstile  103st
1
----------------------------
{'stop_name': '103stcoronaplaza', 'line': '7', 'zipcode': '11368', 'turnstile_station_id': ''}
station  103stcoronaplaza
7
turnstile  103stcorona
7
----------------------------
station  103stcoronaplaza
7
turnstile  103stcorona
7
----------------------------
{'stop_name': '104st', 'line': 'AS', 'zipcode': '11419', 'turnstile_station_id':

{'stop_name': '231st', 'line': '1', 'zipcode': '10463', 'turnstile_station_id': ''}
station  231st
1
turnstile  231st
1
----------------------------
station  231st
1
turnstile  231st
1
----------------------------
{'stop_name': '233st', 'line': '5', 'zipcode': '10466', 'turnstile_station_id': ''}
{'stop_name': '238st', 'line': '1', 'zipcode': '10463', 'turnstile_station_id': ''}
station  238st
1
turnstile  238st
1
----------------------------
station  238st
1
turnstile  238st
1
----------------------------
{'stop_name': '23st', 'line': '466', 'zipcode': '10010', 'turnstile_station_id': ''}
station  23st
466
turnstile  23st
6
----------------------------
station  23st
466
turnstile  23st
6
----------------------------
{'stop_name': '23st', 'line': 'ACE', 'zipcode': '10011', 'turnstile_station_id': ''}
station  23st
ACE
turnstile  23st
CE
----------------------------
station  23st
ACE
turnstile  23st
CE
----------------------------
{'stop_name': '23st', 'line': '2', 'zipcode': '10011', '

turnstile  80st
A
----------------------------
station  80st
AS
turnstile  80st
A
----------------------------
{'stop_name': '81st', 'line': 'ABC', 'zipcode': '10024', 'turnstile_station_id': ''}
station  81st
ABC
turnstile  181st
A
----------------------------
station  81st
ABC
turnstile  181st
A
----------------------------
{'stop_name': '82stjacksonhts', 'line': '7', 'zipcode': '11372', 'turnstile_station_id': ''}
station  82stjacksonhts
7
turnstile  82stjacksonh
7
----------------------------
station  82stjacksonhts
7
turnstile  82stjacksonh
7
----------------------------
{'stop_name': '85stforepky', 'line': 'J', 'zipcode': '11421', 'turnstile_station_id': ''}
station  85stforepky
J
turnstile  85stforestpk
J
----------------------------
station  85stforepky
J
turnstile  85stforestpk
J
----------------------------
{'stop_name': '86st', 'line': '2', 'zipcode': '10024', 'turnstile_station_id': ''}
{'stop_name': '86st', 'line': 'ABC', 'zipcode': '10024', 'turnstile_station_id': ''}
sta

{'stop_name': 'broadwayjunction', 'line': 'AC', 'zipcode': '11233', 'turnstile_station_id': ''}
{'stop_name': 'broadwayjunction', 'line': 'L', 'zipcode': '11207', 'turnstile_station_id': ''}
{'stop_name': 'broadwayjunction', 'line': 'JZ', 'zipcode': '11207', 'turnstile_station_id': ''}
{'stop_name': 'bronxparkea', 'line': '5', 'zipcode': '10462', 'turnstile_station_id': ''}
{'stop_name': 'brookave', 'line': '6', 'zipcode': '10454', 'turnstile_station_id': ''}
station  brookave
6
turnstile  brookav
6
----------------------------
station  brookave
6
turnstile  brookav
6
----------------------------
{'stop_name': 'brooklynbridgecityhall', 'line': '4566', 'zipcode': '10000', 'turnstile_station_id': ''}
{'stop_name': 'brooklyncollegeflatbushave', 'line': '5', 'zipcode': '11210', 'turnstile_station_id': ''}
{'stop_name': 'buhreave', 'line': '66', 'zipcode': '10461', 'turnstile_station_id': ''}
station  buhreave
66
turnstile  buhreav
6
----------------------------
station  buhreave
66
turnsti

station  elmhurave
EMR
turnstile  elmhurstav
MR
----------------------------
station  elmhurave
EMR
turnstile  elmhurstav
MR
----------------------------
{'stop_name': 'essexst', 'line': 'JMZ', 'zipcode': '10002', 'turnstile_station_id': ''}
{'stop_name': 'euclidave', 'line': 'ACS', 'zipcode': '11208', 'turnstile_station_id': ''}
station  euclidave
ACS
turnstile  euclidav
AC
----------------------------
station  euclidave
ACS
turnstile  euclidav
AC
----------------------------
{'stop_name': 'farrockawaymottave', 'line': 'A', 'zipcode': '11691', 'turnstile_station_id': ''}
station  farrockawaymottave
A
turnstile  farrockaway
A
----------------------------
station  farrockawaymottave
A
turnstile  farrockaway
A
----------------------------
{'stop_name': 'flushingmainst', 'line': '77', 'zipcode': '111354', 'turnstile_station_id': ''}
station  flushingmainst
77
turnstile  flushingmain
7
----------------------------
station  flushingmainst
77
turnstile  flushingmain
7
-----------------------

turnstile  jeffersonst
L
----------------------------
{'stop_name': 'junctionblvd', 'line': '77', 'zipcode': '11368', 'turnstile_station_id': ''}
station  junctionblvd
77
turnstile  junctionblvd
7
----------------------------
station  junctionblvd
77
turnstile  junctionblvd
7
----------------------------
{'stop_name': 'juniusst', 'line': '4', 'zipcode': '11207', 'turnstile_station_id': ''}
{'stop_name': 'kewgaensuniontpke', 'line': 'EF', 'zipcode': '11415', 'turnstile_station_id': ''}
{'stop_name': 'kingshwy', 'line': 'F', 'zipcode': '11223', 'turnstile_station_id': ''}
station  kingshwy
F
turnstile  kingshwy
F
----------------------------
station  kingshwy
F
turnstile  kingshwy
F
----------------------------
{'stop_name': 'kingshwy', 'line': 'BQ', 'zipcode': '11229', 'turnstile_station_id': ''}
station  kingshwy
BQ
turnstile  kingshwy
BQ
----------------------------
station  kingshwy
BQ
turnstile  kingshwy
BQ
----------------------------
{'stop_name': 'kingshwy', 'line': 'N', 'zipcode

turnstile  neptuneav
F
----------------------------
station  neptuneave
F
turnstile  neptuneav
F
----------------------------
{'stop_name': 'nereidave238st', 'line': '5', 'zipcode': '10466', 'turnstile_station_id': ''}
{'stop_name': 'nevinsst', 'line': '2345', 'zipcode': '11217', 'turnstile_station_id': ''}
station  nevinsst
2345
turnstile  nevinsst
2345
----------------------------
station  nevinsst
2345
turnstile  nevinsst
2345
----------------------------
{'stop_name': 'newlotsave', 'line': 'L', 'zipcode': '11207', 'turnstile_station_id': ''}
station  newlotsave
L
turnstile  newlots
L
----------------------------
station  newlotsave
L
turnstile  newlots
L
----------------------------
{'stop_name': 'newlotsave', 'line': '4', 'zipcode': '11207', 'turnstile_station_id': ''}
{'stop_name': 'newutrechtave', 'line': 'N', 'zipcode': '11204', 'turnstile_station_id': ''}
{'stop_name': 'newkirkave', 'line': 'BQ', 'zipcode': '11226', 'turnstile_station_id': ''}
station  newkirkave
BQ
turnstile 

{'stop_name': 'unionst', 'line': 'DNR', 'zipcode': '11217', 'turnstile_station_id': ''}
station  unionst
DNR
turnstile  unionst
R
----------------------------
station  unionst
DNR
turnstile  unionst
R
----------------------------
{'stop_name': 'uticaave', 'line': 'AC', 'zipcode': '11207', 'turnstile_station_id': ''}
station  uticaave
AC
turnstile  uticaav
AC
----------------------------
station  uticaave
AC
turnstile  uticaav
AC
----------------------------
{'stop_name': 'vancortlatpark242st', 'line': '1', 'zipcode': '10463', 'turnstile_station_id': ''}
{'stop_name': 'vansiclenave', 'line': 'AC', 'zipcode': '11207', 'turnstile_station_id': ''}
station  vansiclenave
AC
turnstile  vansiclenave
C
----------------------------
station  vansiclenave
AC
turnstile  vansiclenave
C
----------------------------
{'stop_name': 'vansiclenave', 'line': 'JZ', 'zipcode': '11207', 'turnstile_station_id': ''}
station  vansiclenave
JZ
turnstile  vansiclenav
JZ
----------------------------
station  vansicl

In [440]:
print(fuzz.ratio('beach25st','beach36thst'))

70


In [471]:
empty_count = 0
for i in stations_dict:
    if stations_dict[i]['turnstile_station_id'] == '':
            empty_count+=1
            
print(empty_count)

146


In [402]:
columns = (stations_dict[0].keys())
csv_columns = list(columns)

In [403]:
with open('test.csv', 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
    writer.writeheader()
    for key, value in stations_dict.items():
        writer.writerow(value)

In [404]:
new_stations_df = pd.read_csv('test.csv')

In [406]:
new_stations_df.head()

,stop_name,line,zipcode,turnstile_station_id
0,103rdst,ABC,10025-4403,R314
1,103rdst,466,10029,R180
2,103rdst,1,10025,R191
3,103rdstcoronaplaza,7,11368,R208
4,104thst,AS,11419,R354


In [408]:
new_stations_df.turnstile_station_id.value_counts()#[1:20]

R358    5
R034    4
R354    4
R344    3
R285    3
       ..
R460    1
R303    1
R271    1
R185    1
R186    1
Name: turnstile_station_id, Length: 281, dtype: int64

In [409]:
new_stations_df[new_stations_df.turnstile_station_id=='R358']

,stop_name,line,zipcode,turnstile_station_id
187,beach25thst,A,11691,R358
188,beach36thst,A,11692,R358
189,beach44thst,A,11691,R358
190,beach60thst,A,11692,R358
191,beach67thst,A,12692,R358
